In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ~/input.txt

--2024-10-04 08:59:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘/Users/josemiguelvilchesfierro/input.txt’

/Users/josemiguelvi 100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-10-04 08:59:17 (10.6 MB/s) - ‘/Users/josemiguelvilchesfierro/input.txt’ saved [1115394/1115394]



In [4]:
# read it in to inspect it
with open('/Users/josemiguelvilchesfierro/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
#Hyperparameters
batch_size = 32 # number of sequences in a mini-batch
block_size = 8  # number of characters in a sequence
max_iter = 3000 # number of training iterations
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
n_emb = 32

In [6]:
list_ch = sorted(list(set(text)))
vocab_size = len(list_ch)
stoi = {ch:i for i,ch in enumerate(list_ch)}
itos = {i:ch for i,ch in enumerate(list_ch)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda h: ''.join([itos[i] for i in h])

In [7]:
#Encode dataset
text_encode = torch.tensor(encode(text), dtype=torch.long)
#Create a dataset
n = int(len(text_encode) * 0.9)
train = text_encode[:n]
valid = text_encode[n:]

In [71]:
torch.manual_seed(1337)
def get_batch(split):
    data = train if split == 'train' else valid
    ix = torch.randint(len(data)- block_size, (batch_size,))
    X = torch.stack([data[   i : i+block_size   ] for i in ix])
    Y = torch.stack([data[i + 1: i+block_size +1] for i in ix])
    return X, Y

In [72]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [118]:
class BigramLanguajeModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.table_embedding_token = nn.Embedding(vocab_size, n_emb)
        self.table_embedding_position = nn.Embedding(block_size, n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size) 

    def forward(self, idx, targets= None):
        B, T = idx.shape  
        #Embedding tokens and targets in a 3d tensor
        emb = self.table_embedding_token(idx) # B T C [4x 8 x 32]
        pos = self.table_embedding_position(torch.arange(T, device=idx.device))  # [T, n_emb]
        X = emb + pos
        #Linear layer
        logits = self.lm_head(X) # B T C [4 x 8 x 65]

        
        if targets is None:
            loss = None 
        else:
            B , T, C = logits.shape
            #Reshape the tensor to 2d for cross entropy
            logits = logits.view(B*T , C) #[32 x 65]
            targets = targets.view(B*T) #[32]
            #Calculate loss
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            T = idx.shape[1]
            #get the prediction
            logits, _ = self(idx) # [B x T x C] because targets is None [4 x 8 x 65]
            #get the last token
            logits = logits[:, -1, :] # Becomes [B x C] [4 x 65]
            #softmax
            probs = F.softmax(logits, dim=1) 
            #sample
            new_token = torch.multinomial(probs, 1)
            #append to the sequence
            idx = torch.cat([idx, new_token], dim=1) # B x T+1   

        return idx

In [119]:
model = BigramLanguajeModel()

In [120]:
optimier = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [121]:
for i in range(max_iter):

    # every once in a while evaluate the loss on train and val sets
    if (i % eval_interval == 0) or (i == max_iter - 1):
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #forward step
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)


    # Print the loss every 1000 epochs
    #if i % 1000 == 0:
    #    print(f'Epoch [{i}/{max_iter}], Loss: {loss.item():.4f}')
    
    #backward step
    optimier.zero_grad(set_to_none=True)
    loss.backward()
    optimier.step()


step <built-in function iter>: train loss 4.5408, val loss 4.5383
step <built-in function iter>: train loss 2.5510, val loss 2.5584
step <built-in function iter>: train loss 2.5237, val loss 2.5354
step <built-in function iter>: train loss 2.5041, val loss 2.5194
step <built-in function iter>: train loss 2.4949, val loss 2.5309
step <built-in function iter>: train loss 2.4965, val loss 2.5298
step <built-in function iter>: train loss 2.4958, val loss 2.5232
step <built-in function iter>: train loss 2.5041, val loss 2.5273
step <built-in function iter>: train loss 2.4843, val loss 2.5244
step <built-in function iter>: train loss 2.5003, val loss 2.5112
step <built-in function iter>: train loss 2.4901, val loss 2.5186


In [129]:
model = BigramLanguajeModel()
idx = torch.zeros((1,1), dtype=torch.long)
generate = model.generate(idx, 8)[0].tolist()
print(decode(generate))


dSPr-Cwy


In [35]:
X, Y = get_batch('train')
token_embedding_table = nn.Embedding(vocab_size, n_emb)
token_emb = token_embedding_table(X)

torch.Size([32, 8, 32])

In [33]:
T = X.shape[1]
position_embedding_table = nn.Embedding(block_size, n_emb)
pos_emb = position_embedding_table(torch.arange(T))

In [39]:
pos_emb.shape

torch.Size([8, 32])

In [37]:
(token_emb + pos_emb).shape

torch.Size([32, 8, 32])